In [11]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation



In [12]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
batchsize = 8



In [14]:
train_datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=0.2, shear_range=0.2,
                                   zoom_range=0.2, width_shift_range=0.2,
                                   height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    r'/Users/kronos27/Desktop/ProjectData/color_data/train',
    target_size=(80, 80), batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(
    r'/Users/kronos27/Desktop/ProjectData/color_data/train',
    target_size=(80, 80), batch_size=batchsize, class_mode='categorical', subset='validation')



Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [16]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_data = test_datagen.flow_from_directory(r'/Users/kronos27/Desktop/ProjectData/color_data/test',
                                             target_size=(80, 80), batch_size=batchsize, class_mode='categorical')



Found 0 images belonging to 0 classes.


In [15]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False



In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint


In [ ]:

checkpoint = ModelCheckpoint(r'/Users/kronos27/Desktop/ProjectData/models',
                             monitor='val_loss', save_best_only=True, verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3, )

callbacks = [checkpoint, earlystop, learning_rate]



In [ ]:
model.compile(optimizer='Sgd', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_data, steps_per_epoch=train_data.samples // batchsize,
                    validation_data=validation_data,
                    validation_steps=validation_data.samples // batchsize,
                    callbacks=callbacks,
                    epochs=5)



In [ ]:
loss_tr, acc_tr =  model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
loss_vr, acc_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
loss_test, acc_test = model.evaluate(test_data)
print(loss_test)
print(acc_test)